# First chat

First part of the chain, just setup api key (on `azure` in this example). This cell prints the name of the base deployment just to verify that we correctly loaded the .env file.

Remember you need a .env file with the following content:

```
OPENAI_API_KEY=xxxxxxxxx
OPENAI_API_BASE=https://alkopenai2.openai.azure.com/
HUGGINGFACEHUB_API_TOKEN=xxxxxxx
PINECONE_KEY=xxxxxxx
PINECONE_ENV=us-west1-gcp-free
SERPAPI_API_KEY=xxxxx
```

In [1]:
import os
import openai
from pprint import pprint

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_type = "azure"
openai.api_version = "2023-03-15-preview"

# Remember that you need to set the OPENAI_API_BASE to point openai to your specific deployment.

openai.api_base = os.getenv('OPENAI_API_BASE')
openai.api_key = os.getenv("OPENAI_API_KEY")

# print base to verify you are pointint to the right deployment
print(openai.api_base)


https://alkopenai2.openai.azure.com/


As always I'm creating various llm 

In [16]:
from langchain.chat_models import AzureChatOpenAI
from langchain.llms import AzureOpenAI  

# Pay attention that gpt35 and gpt4 are chat based llms
gpt35 =  AzureChatOpenAI(
    openai_api_version="2023-03-15-preview",
    deployment_name="Gpt35", 
    model_name="gpt-35-turbo"
)

gpt4 =  AzureChatOpenAI(
    openai_api_version="2023-03-15-preview",
    deployment_name="Gpt4", 
    model_name="gpt-4"
)

# this is not a Chat model this is a text model
davinci = AzureOpenAI(
	temperature=0,
	openai_api_version="2023-03-15-preview",
    deployment_name="text-davinci-003", 
	model_name="text-davinci-003"
)

Now the example starts to behave differently, we starts using a real `chat object` from langchain.

In [22]:
from langchain.chains import ConversationChain
from pprint import pprint

conversation = ConversationChain(
    llm=davinci, # use davinci that is not a real chat model but it simply works.
    verbose=False #Feel free to set to true to look for internal working, in this example is not necessary
)

# Actually the conversation has some actual memory
pprint(f"Memory for the conversation is {conversation.memory}")

result = conversation.predict(input = "Hi my name is Gian Maria, what is your name?")
pprint(result)
pprint("--------------------------")
result = conversation.predict(input= "What is my name?")
pprint(result)

('Memory for the conversation is chat_memory=ChatMessageHistory(messages=[]) '
 "output_key=None input_key=None return_messages=False human_prefix='Human' "
 "ai_prefix='AI' memory_key='history'")
" Hi Gian Maria, my name is AI-1. It's nice to meet you."
'--------------------------'
' Your name is Gian Maria.'


Now we verified that the conversation has some memory, this means that the calls to the LLM is done using some form of memory. Now lets try **to dump some memory content to verify what is inside it**. As you can verify it contains simply the list of the messages, like previous example.

In [26]:
pprint (conversation.memory.chat_memory.messages)

[HumanMessage(content='Hi my name is Gian Maria, what is your name?', additional_kwargs={}, example=False),
 AIMessage(content=" Hi Gian Maria, my name is AI-1. It's nice to meet you.", additional_kwargs={}, example=False),
 HumanMessage(content='What is my name?', additional_kwargs={}, example=False),
 AIMessage(content=' Your name is Gian Maria.', additional_kwargs={}, example=False)]


Now lets try to use a different form of `memory called ConversationBufferMemory`.

In [82]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=davinci, # use davinci that is not a real chat model but it simply works.
    memory=memory,
    verbose=False #Feel free to set to true to look for internal working, in this example is not necessary
)

result = conversation.predict(input = "Hi my name is Gian Maria, what is your name?")
pprint(result)
pprint("--------------------------")
result = conversation.predict(input= "What is my name?")
pprint(result)

" Hi Gian Maria, my name is AI-1. It's nice to meet you."
'--------------------------'
' Your name is Gian Maria.'


Result is pretty much the same, it seems that the ConverstaionBufferMemory works similarly to the ChatMessageHistory form of memory.

In [68]:
pprint (memory) #Please verify output, you can see that this wraps a ChatMessageHistory.
pprint("------------------")
pprint (memory.chat_memory.messages)
pprint("------------------")
pprint (memory.buffer)


ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Hi my name is Gian Maria, what is your name?', additional_kwargs={}, example=False), AIMessage(content=" Hi Gian Maria, my name is AI-1. It's nice to meet you.", additional_kwargs={}, example=False), HumanMessage(content='What is my name?', additional_kwargs={}, example=False), AIMessage(content=' Your name is Gian Maria.', additional_kwargs={}, example=False)]), output_key=None, input_key=None, return_messages=False, human_prefix='Human', ai_prefix='AI', memory_key='history')
'------------------'
[HumanMessage(content='Hi my name is Gian Maria, what is your name?', additional_kwargs={}, example=False),
 AIMessage(content=" Hi Gian Maria, my name is AI-1. It's nice to meet you.", additional_kwargs={}, example=False),
 HumanMessage(content='What is my name?', additional_kwargs={}, example=False),
 AIMessage(content=' Your name is Gian Maria.', additional_kwargs={}, example=False)]
'------------------

Why we want to use a memory explicitly **instead of avoiding using the default one?**. One of the answer is the ability to tune the chat memory manually.

In [69]:
# you can get the history
history =memory.load_memory_variables({})
pprint (history)


{'history': 'Human: Hi my name is Gian Maria, what is your name?\n'
            "AI:  Hi Gian Maria, my name is AI-1. It's nice to meet you.\n"
            'Human: What is my name?\n'
            'AI:  Your name is Gian Maria.'}


In [71]:
# i can reset the memory
memory.clear()
result = conversation.predict(input= "What is my name?")
pprint(result)
pprint(memory.buffer)

' Your name is John.'
'Human: What is my name?\nAI:  Your name is John.'


In [92]:
memory.clear()

# I can preload the memory with a conversation, so we can start from a known state
memory.save_context({"input" : "Hy my name is Gian Maria"}, {"output" : "Hi Gian Maria my name is Alfred!"})
pprint(memory.buffer)
pprint (memory.chat_memory.messages)

print ("------------------")
result = conversation.predict(input= "What is my name? What is your name?")
pprint(result)


'Human: Hy my name is Gian Maria\nAI: Hi Gian Maria my name is Alfred!'
[HumanMessage(content='Hy my name is Gian Maria', additional_kwargs={}, example=False),
 AIMessage(content='Hi Gian Maria my name is Alfred!', additional_kwargs={}, example=False)]
------------------
'Your name is Gian Maria, and my name is Alfred.'


In [88]:
result = conversation.predict(input= "Sum the number of common letters of our names, give a detailed explanation of the answer.")
pprint(result)

('I apologize for my previous response. Let me correct myself. The total '
 'number of common letters in our names is 4. The unique common letters are '
 "'a', 'r', 'i', and 'm'. Gian Maria and Alfred share these four letters.")


One of the cool stuff for this kind of interaction is that we can change anything **as an example I can change llm**

In [95]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    conversation.llm = gpt4 ##try using davinci
    conversation.verbose = True
    result = conversation.predict(input= "Sum the number of common letters of our names, give a detailed explanation of the answer.")
    pprint(result)
    print(cb)

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..




> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hy my name is Gian Maria
AI: Hi Gian Maria my name is Alfred!
Human: What is my name? What is your name?
AI: Your name is Gian Maria, and my name is Alfred.
Human: Sum the number of common letters of our names, give a detailed explanation of the answer.
AI: To sum the number of common letters in our names, we first need to list the unique letters in each of our names and then compare them to find the common ones. Here is the breakdown:

Your name is Gian Maria:
G, I, A, N, M, R

My name is Alfred:
A, L, F, R, E, D

Now let's compare the unique letters of both names and find the common ones:

Common letters: A, R

There are 2 common letters between our names, which are A and R.
H